### Use Huayi's time binning function to assign our geotagged tweets to 15-minute time bins:
#### 05, 20, 35, and 50 mins past the hour. 

In [21]:
import pandas as pd
import numpy as np
import datetime as dt

In [22]:
# Huayi's time binning function! 

from datetime import timedelta 

def binMinute(minute):
    '''
    Calculate which of the 15min bins the current minute falls in.
    The bins are 05, 20, 35, 50. 
    Current minutes less than 05 belong to bin 50 of the hour before. 
    '''    
    m2q= (minute-5)/15
    if m2q < 0:
        newminute = -10 
    elif m2q < 1:
        newminute = 5
    elif m2q < 2:
        newminute = 20
    elif m2q < 3:
        newminute = 35
    elif m2q < 4:
        newminute = 50
    return newminute       
    
def BinTime(df_datetime_col):
    '''
    Take a df column that contains datetime object (df_datetime_col)
    Return a new column that contains the binned datetime object
    The bins are as defined in binMinute
    '''
    binned_col= df_datetime_col.apply(lambda x: x.replace(minute = 0) + timedelta(minutes=binMinute(x.minute)))
    return binned_col

In [23]:
df = pd.read_pickle("../data/02_geotweets_labeled_ee.pkl")

In [24]:
# df.head()

In [25]:
df['time_bin'] = BinTime(df.date.dt.floor('min'))

In [26]:
# df.head()

In [27]:
df = df.reset_index(drop=True)
# df.head()

In [28]:
# Localize time bin (remove comparison to UTC)

time_bin = []
for i in range(0, len(df)):
    bn = df.iloc[i].time_bin.tz_convert(None) + pd.Timedelta(-4,'h')
    time_bin.append(bn)

df['time_bin'] = time_bin

In [29]:
df.head()

,date,text,username,day_of_week,hour,safe,time_bin
0,2020-03-23 14:10:14-04:00,𝐐𝐔𝐄𝐒𝐓𝐈𝐎𝐍 | Are you moving for thirty minutes e...,lifenleggings,Monday,14,2,2020-03-23 14:05:00
1,2020-03-23 15:03:32-04:00,"#newnormal Day 8 Oh dayum, did Governor #andre...",miles_randy,Monday,15,1,2020-03-23 14:50:00
2,2020-03-23 18:14:17-04:00,#isolation #prospectpark #prospectparkalliance...,andrewbwhite1,Monday,18,2,2020-03-23 18:05:00
3,2020-03-23 18:15:25-04:00,#isolation #prospectpark #fishing #prospectpar...,andrewbwhite1,Monday,18,2,2020-03-23 18:05:00
4,2020-03-23 18:17:12-04:00,#isolation #prospectpark #prospectparkalliance...,andrewbwhite1,Monday,18,2,2020-03-23 18:05:00


In [30]:
len(df)

364

In [10]:
df.to_pickle("../data/02_geotweets_labeled_binned_ee.pkl")